## Step 4:  Growth Rate and Allostery Analysis

This notebook imports a series of mutant counts (already corrected for sequencing error by a Hamming filter), and computes relative growth rates and allosteric effects. The data is then saved as .txt files and pickle databases. 

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
import pandas as pd
import pickle 
from scipy import stats 
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
from Bio.SeqUtils import seq3
from Bio.SeqUtils import seq1
from matplotlib import rcParams
import csv
import itertools
from Bio.PDB.PDBParser import PDBParser
from pathlib import Path #introduced in Python 3.4

## 0. Initialize experiment parameters

In [2]:
#the wt sequence of DHFR
dhfr_wt = 'MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHTWESIGRPLPGRKNIILSSQPGTDDRVTWVKSVDEAIAACGDVPEIMVIGGGRVYEQFLPKAQKLYLTHIDAEVEGDTHFPDYEPDDWESVFSEFHDADAQNSHSYCFEILERR'
dhfr_wt_ix = np.arange(len(dhfr_wt[:]))

#all amino acids
aas = 'ACDEFGHIKLMNPQRSTVWY*'
aas_ix = np.arange(len(aas[:]))

#the experiment
SL_comp = [list(range(40)),list(range(40,80)),list(range(80,120)),list(range(120,159))] #cutoffs between sublibraries. e.g. SL1=range(0,40)
SL_name = ['SL1','SL2','SL3','SL4'] #names of sublibraries 
vials = ['V1','V2','V3','V4','V5','V6']  #the six vial positions in the turbidostat
lit_vials = ['V1','V2','V3'] #these vials were lit during the experiments
dark_vials = ['V4','V5','V6'] #these vials were kept in the dark
vial_timepoints = [1,2,3,4,5,6] #number of timepoints
act_time = [0, 4, 8, 12, 16, 20, 24]#hours from start of experiment that the timepoints were taken

#dimensions of stuff
nTP = len(act_time)
nSL = len(SL_comp)
n_pos = len(dhfr_wt)
n_aa = len(aas)
n_vials = len(vials)
n_rpts = len(lit_vials)

## 1. Data Import, Growth Rate fitting, and computing allosteric effects

**Importing all of the data from text files.** Each vial is represented as a three dimensional matrix of 159(pos)\*21(aa)\*6(timepoints). Vials 1-3 are repeats in the light, Vials 4-6 are repeats in the dark. All six vials share a single t=0 time point (were inoculated from the same starting culture). The set of six 3D matrices then is compiled into a single dictionary, containing all the raw counts. <br>
We also create a matrix of WT counts, which is 4(sublibraries) * 6(timepoints); this gets used in future normalization steps. Data were previously processed by the script: [insert script name(s) here] 

In [3]:
file_suffix='hard_adjusted_residue.txt' 
vial_count_matrices = {}
WT_counts = np.zeros((nSL,nTP)) #matrix for storing the WT values for each sample (a vial*timepoint combo)
path_to_input = Path("../Step_3_hamming_filtering/output/") #OS agnostic pathing

#load t=0 (shared for all vials)
t0_counts = np.zeros((n_pos,n_aa))#make the array to store the counts in the T0 timepoint
f_name_t0 = 'T0'+file_suffix 
filepath_t0 = path_to_input / f_name_t0 #set up absolute path so we can go to previous output
flines = open(filepath_t0,'r').readlines()
for line in flines:
    sp_line = line.split()
    if sp_line[1] == 'WT':
        SLix = SL_name.index(sp_line[0])
        WT_counts[SLix,0] = float(sp_line[2])
    elif sp_line[1] == 'SNEAKY_WT': #silent mutations that encode wildtype, they are thrown out here 
        continue
    else:
        pos_ix = int(sp_line[1][1:-1])-1
        res_ix = aas.index(sp_line[1][-1])
        counts = 0 if float(sp_line[2]) < 1 else float(sp_line[2])
        t0_counts[pos_ix,res_ix] = counts     

#load all other timepoints
for v in vials:
    vial_counts = np.zeros((n_pos,n_aa,nTP))
    vial_counts[:,:,0] = t0_counts
    for tp_ix in vial_timepoints:
        f_name = 'T'+str(tp_ix)+v+file_suffix
        filepath_timepoint = path_to_input / f_name
        flines = open(filepath_timepoint,'r').readlines()
        for line in flines:
            sp_line = line.split()
            if sp_line[1] == 'WT':
                SLix = SL_name.index(sp_line[0])
                WT_counts[SLix,tp_ix] = float(sp_line[2])
            elif sp_line[1] != 'SNEAKY_WT': #silent mutations that encode wildtype, they are thrown out here 
                pos_ix = int(sp_line[1][1:-1])-1
                res_ix = aas.index(sp_line[1][-1])
                counts = 0 if float(sp_line[2]) < 1 else float(sp_line[2])
                vial_counts[pos_ix,res_ix,tp_ix] = counts
    vial_count_matrices[v] = vial_counts 
    
#go back through and fill in WT values in all matrices
#this makes sure that all zero values in the matrix truly indicate missing data/absent reads
for v in vials:
    for p in range(n_pos):
        SLix = [p in sl for sl in SL_comp].index(True)
        wt_aa_ix = aas.index(dhfr_wt[p])
        for tp_ix in range(nTP):
            vial_count_matrices[v][p,wt_aa_ix,tp_ix] = WT_counts[SLix,tp_ix]

**Normalize the counts to produce relative frequencies**. <br>Produce another set of numpy matrices in which all counts are normalized to a relative frequency $f_a$ as follows: <br>
$ f_a = log(\frac{f_a}{f_{WT}})_{t} - log(\frac{f_a}{f_{WT}})_{t=0}$

<br>
Note that this code will produce some warnings, because all mutations with zero counts will produce a $-\infty$ upon division.

In [4]:
vial_count_norm_mat = {}
for v in vials:
    vial_counts_norm = np.zeros(vial_count_matrices[v].shape)
    for p in range(n_pos):
        SLix = [p in sl for sl in SL_comp].index(True)
        for tp_ix in range(nTP):
            vial_counts_norm[p,:,tp_ix] = np.log(vial_count_matrices[v][p,:,tp_ix]/WT_counts[SLix,tp_ix]) - \
                                            np.log(vial_count_matrices[v][p,:,0]/WT_counts[SLix,0])
    vial_count_norm_mat[v] = vial_counts_norm
    #2 warnings are expected: divide by zero, and invalid value. 

C:\Users\s178464\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
C:\Users\s178464\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in subtract
  import sys


# **Compute growth rates.**  
 We now compute relative growth rates from the matrix of relative frequencies. These growth rates are stored as two three dimensional matrices, of size(159(pos)\*21(aa)\*3(repeats)). There is one 3D matrix containing all of the "lit" replicates; and one containing all the "dark" replicates. <br>

Growth rates are computed by linear regression of $f_a$ vs time. In order to fit a growth rate, there must be > T0_gate counts present at t=0. Growth rate is only fit for data points up to the first point where counts go to zero ($f_a = -\infty $), with a minimum of four points required for fitting.

Alongside these two matrices, we create a second pair of "masking" matrices, which encode information about missing data and WT positions. More specifically, 0==position ok, 1==position missing at t=0, 2==position present at t=0, but not at sufficient quantities to allow fitting later on, 3==WT position

Finally, we condense the data for all replicates into a set of average growth rates (and associated standard deviations) in the lit and dark. Here, we only average over points where fitting was possible ("0" in the masking matrix). If computing an average growth rate is not possible (none of the repeats had sufficient data to permit fitting) we mark that position as having a relative growth rate of -1000 (which should never actually occur in these data). In the case that only one repeat permits fitting, we take this value as the average (here it is less clear what to do about the SD... for now I leave it zero). Also record a matrix of the number of points used for averaging for both lit and dark growth rates (to use in assessing variance later).

In [5]:
#First, some parameter and function definitions.
T0gate = 50

#function for computing growth rates. 
#input parameters:
#mat_counts= a npos*naa*ntimepoints matrix of raw counts, 
#mat_normcounts = associated npos*naa*ntimepoints matrix of normalized counts.
#returns:
#grates = a npos*naa matrix of fit growth rates, 
#mask = an associated npos*naa "mask" matrix
def fit_gr_all(mat_counts,mat_normcounts):
    grates = np.zeros([n_pos,n_aa])
    mask = np.zeros([n_pos,n_aa])
    for p in range(n_pos):
        for a,res in enumerate(aas):
            #was if mat_counts[p,a,0] <= T0gate:
            if mat_counts[p,a,0] < T0gate:
                if mat_counts[p,a,0] < T0gate and mat_counts[p,a,0] > 0:
                    mask[p,a] = 2
                else:
                    mask[p,a] = 1
            else:
                #checking to see which timepoints have more than zero counts
                if (False in list(mat_counts[p,a,:] > 1)):
                    stop_fit_ix = list(mat_counts[p,a,:] > 1).index(False) 
                else:
                    stop_fit_ix = len(mat_counts[p,a,:])
                    
                if stop_fit_ix < 3:
                    #not enough data to fit
                    mask[p,a] = 2
                else:
                    m,b,r,pval,se = stats.linregress(act_time[0:stop_fit_ix],mat_normcounts[p,a,0:stop_fit_ix])
                    grates[p,a] = m
            if res == dhfr_wt[p]:
                mask[p,a] = 3          
    return grates,mask

#function for averaging growth rates across repeats.
#input parameters:
#gr_mat = npos*naa*nrpts matrix of growth rates
#mask_mat = associated npos*naa*nrpts masking matrix (that we use to determine which points to average)
#the function returns:
#avg_mat = npos*naa matrix of average growth rates
#std_mat = npos*naa matrix of std deviations in growth rates
#n_samp_mat = npos*naa matrix of number of measurements used to compute each average
def avg_gr_replicates(gr_mat,mask_mat):
    avg_mat,std_mat,n_samp_mat = np.zeros([n_pos,n_aa]),np.zeros([n_pos,n_aa]),np.zeros([n_pos,n_aa])
    for p in range(n_pos):
        for a in range(n_aa):
            to_avg = [gr_mat[p,a,k] for k,mval in enumerate(mask_mat[p,a]) if mval==0]
            n_samp_mat[p,a]=len(to_avg)
            if len(to_avg) == 0:
                to_avg_quick_death = [gr_mat[p,a,k] for k,mval in enumerate(mask_mat[p,a]) if mval==2]
                
                #changing this from <= 1 to <= 0, aka = 0. 
                if len(to_avg_quick_death) <= 0:
                    avg_mat[p,a] = -1000
                else: avg_mat[p,a] = -999   
            elif len(to_avg) == 1:
                #here changed to -999 from to_avg[0]
                avg_mat[p,a] = -999
            else:
                avg_mat[p,a] = np.mean(to_avg)
                std_mat[p,a] = np.std(to_avg)
    return avg_mat,std_mat,n_samp_mat


In [6]:
#The actual calculations.
#compute the growth rates and masks for all lit and dark vials
all_gr_lit, all_gr_dark = np.zeros([n_pos,n_aa,n_rpts]), np.zeros([n_pos,n_aa,n_rpts])
all_gr_lit_mask, all_gr_dark_mask = np.zeros([n_pos,n_aa,n_rpts]), np.zeros([n_pos,n_aa,n_rpts])
for i,v in enumerate(lit_vials):
    gr_tmp,mask_tmp = fit_gr_all(vial_count_matrices[v],vial_count_norm_mat[v])
    all_gr_lit[:,:,i] = gr_tmp
    all_gr_lit_mask[:,:,i] = mask_tmp
    
for i,v in enumerate(dark_vials):
    gr_tmp,mask_tmp = fit_gr_all(vial_count_matrices[v],vial_count_norm_mat[v])
    all_gr_dark[:,:,i] = gr_tmp
    all_gr_dark_mask[:,:,i] = mask_tmp
    
#compute average and std error across repeats for each condition
avg_gr_lit, sd_gr_lit, n_pts_gr_lit = avg_gr_replicates(all_gr_lit,all_gr_lit_mask)
avg_gr_dark, sd_gr_dark, n_pts_gr_lit = avg_gr_replicates(all_gr_dark,all_gr_dark_mask)

**Compute allostery** Given the above set of growth rates, we can now compute the allosteric effect as the difference between the average growth rate in the light, and the average growth rate in the dark. <br>

To indicate mutations where allostery cannot be reliably computed (because one or both average growth rates could not be computed), we mark the allosteric effect as -1000 (which is otherwise not possible in this dataset)

As a measure of the significance of this difference, we also compute a p-value by t-test, which tells us the probability that the means come from the same distribution (using Welch's t-test, which does not assume an equal variance in both samples)

In [7]:
allostery = avg_gr_lit - avg_gr_dark
allostery_pvals = np.ones([n_pos,n_aa])
allostery_sd = np.zeros([n_pos,n_aa])

#In this loop:
#1) set allostery = -1000 for mutations lacking growth rate data
#2) calculate pvals (by Welch's t-test)
#3) calculate propagated error (given the underlying growth rate data)
for p in range(n_pos):
    for a in range(n_aa):

        if (avg_gr_lit[p,a] == -1000) or (avg_gr_dark[p,a] == -1000):
            allostery[p,a] = -1000 
            #These values are from DL121D27N, which is considered catalytically dead.
        if ((avg_gr_lit[p,a] <= -0.1301) or (avg_gr_dark[p,a] <= -0.1329)) and not\
        ((avg_gr_lit[p,a] == -1000) or (avg_gr_dark[p,a] == -1000)):
            allostery[p,a] = -999
            
        lit_vals = [all_gr_lit[p,a,k] for k,mval in enumerate(all_gr_lit_mask[p,a]) if mval==0]
        dark_vals = [all_gr_dark[p,a,k] for k,mval in enumerate(all_gr_dark_mask[p,a]) if mval==0]
        if len(lit_vals)>1 and len(dark_vals) > 1:
            t,pval = stats.ttest_ind(lit_vals,dark_vals,equal_var=False)
            allostery_pvals[p,a] = pval
        
        allostery_sd[p,a] = (sd_gr_lit[p,a]**2 + sd_gr_dark[p,a]**2)**0.5
        
        
#we do the same but without filtering out the catalytically dead mutations        

allostery_all = avg_gr_lit - avg_gr_dark
allostery_pvals_all = np.ones([n_pos,n_aa])
allostery_sd_all = np.zeros([n_pos,n_aa])

for p in range(n_pos):
    for a in range(n_aa):

        if (avg_gr_lit[p,a] == -1000) or (avg_gr_dark[p,a] == -1000):
            allostery_all[p,a] = -1000 
        if ((avg_gr_lit[p,a] <= -999.0) or (avg_gr_dark[p,a] <= -999.0)) and not\
        ((avg_gr_lit[p,a] == -1000) or (avg_gr_dark[p,a] == -1000)):
            allostery_all[p,a] = -999
            
        lit_vals_all = [all_gr_lit[p,a,k] for k,mval in enumerate(all_gr_lit_mask[p,a]) if mval==0]
        dark_vals_all = [all_gr_dark[p,a,k] for k,mval in enumerate(all_gr_dark_mask[p,a]) if mval==0]
        if len(lit_vals)>1 and len(dark_vals) > 1:
            t,pval_all = stats.ttest_ind(lit_vals_all,dark_vals_all,equal_var=False)
            allostery_pvals_all[p,a] = pval_all
        
        allostery_sd_all[p,a] = (sd_gr_lit[p,a]**2 + sd_gr_dark[p,a]**2)**0.5        

### **This concludes the calculations portion of the code**. 
Writing all results to disk.

In [8]:
def picklefiler(variable_name, filename):
    outfile = open(filename,'wb')
    pickle.dump(variable_name,outfile)
    outfile.close()

In [9]:
for ix,v in enumerate(lit_vials): #growth rate for each lit vial separately 
    np.savetxt('output/'+v+'.txt', all_gr_lit[:,:,ix] ,delimiter=',') #V1, V2, V3
    
picklefiler(all_gr_lit,'output/all_gr_lit.pkl') #growth rates for individual replicates in light

for ix,v in enumerate(dark_vials): #growth rate for each dark vial separately 
    np.savetxt('output/'+v+'.txt', all_gr_dark[:,:,ix] ,delimiter=',') #V4, V5, V6
    
picklefiler(all_gr_dark,'output/all_gr_dark.pkl')#growth rates for individual replicates in dark  

picklefiler(t0_counts,'output/t0_counts.pkl')#counts at t=0 

np.savetxt('output/lit_gr_avg.txt', avg_gr_lit, delimiter=',')  
picklefiler(avg_gr_lit,'output/avg_gr_lit.pkl') #average growth rate in the light for all mutations

np.savetxt('output/lit_gr_std.txt', sd_gr_lit, delimiter=',') 
picklefiler(sd_gr_lit,'output/sd_gr_lit.pkl')#standard deviation of the growth rate in the light

np.savetxt('output/dark_gr_avg.txt', avg_gr_dark, delimiter=',') 
picklefiler(avg_gr_dark,'output/avg_gr_dark.pkl')#average growth rate in the dark for all mutations

np.savetxt('output/dark_gr_std.txt', sd_gr_dark, delimiter=',') 
picklefiler(sd_gr_dark,'output/sd_gr_dark.pkl')#standard deviation of the growth rate in the dark

np.savetxt('output/allostery.txt', allostery, delimiter=',') 
picklefiler(allostery,'output/allostery.pkl')#allostery (lit-dark growth rate)
picklefiler(allostery_all,'output/allostery_all.pkl')#allostery not filtering catalytically dead (lit-dark growth rate)

np.savetxt('output/allostery_pvals.txt', allostery_pvals, delimiter=',') 
picklefiler(allostery_pvals,'output/allostery_pvals.pkl')#allostery p-value
picklefiler(allostery_pvals_all,'output/allostery_pvals_all.pkl')#allostery p-value not filtering catalytically dead

np.savetxt('output/allostery_sd.txt', allostery_sd, delimiter=',')
picklefiler(allostery_sd,'output/allostery_sd.pkl')#allostery std deviation  
picklefiler(allostery_sd_all,'output/allostery_sd_all.pkl')#allostery std deviation not filtering catalytically dead

#dictionary of relative frequency of each mutant at each timepoint and vial
picklefiler(vial_count_norm_mat,'output/vial_count_norm_mat.pkl')